In [1]:
## import the necessary packages
import numpy as np
import pandas as pd
from sklearn import datasets
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

**1. Load the data**

In [2]:
data=pd.read_excel("default of credit card clients.xls",header=1)
print (data.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Amen/Desktop/default of credit card clients.xls'

The dataset has 30000 obervations and 25 variables.

We start by looking to the data features (default by first 5 rows).

In [ ]:
data.head()

Checking missing values in variables.

In [ ]:
data.isnull().sum()

Based on the data description and table shows above, there is no missing or nan values in our data set.

**2. Distinguish categorical and continuous variables**

We want to distinguish categorical and continuous variables. When unique value in a variable greater than 15, we consider as continuous variables otherwise we consider as categorical variable. Seperating continuous and categorical variables will help us to summary statistics better. 

In [ ]:
cat_v = []
con_v = []
for c in data.columns:
    if len(data[c].value_counts().index)<=15:
        cat_v.append(c)
    else:
        con_v.append(c)

In [ ]:
print("The continuous variables: ", con_v, "\n")
print("The categorical variables: ", cat_v)

**3. Basic feature analysis**

Let's look more details to the data set.

i. continuous variables

we want to analyst continuous variables.

In [ ]:
data[con_v].describe()

The dataset contains 30,000 credit card clients.

The mean of credit card balance limit is 167,484. The standard deviation is extremely large and unusual, we noticed that the max value is 1 millions that is much larger than Q3.

Average age is around 36 year old with standard deviation of 9.2. Minimum age is 21 and maximum age is 79.

Other variables are people month bills and month payments.

ii.categorical variables

Now, let's check categorical variables.

In [ ]:
pd.DataFrame(data[cat_v], dtype='object').describe()

More than half clients are women.

Education level is mostly university.

Half of clients are single.

And of couse, most of clients have no payment issues.

**4.Visualization**

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
data["default payment next month"].plot(kind="hist")
plt.xticks(range(0,2,1))
plt.xlabel("Payment=0,Not Default Payment=1")
plt.title("Default payment next month")
plt.show()

Based on graph and describe,there are 23364 clients without default payment and 6639 clients with default payment in 2005. Also, base on the graph variable "default payment next month" as our target variable has not a large unbalance values.

In [ ]:
data["default payment next month"].value_counts(normalize=True)

22.12% card holder are default payment and 77.88% card horlder without default payment.

**Credit Card limitation distribution**

In [ ]:
plt.figure(figsize = (12,5))
plt.title('Amount of credit limit')
sns.distplot(data['LIMIT_BAL'],kde=True,bins=200, color="blue")
plt.xticks(range(0,1000000,250000))
plt.ylabel('Percentage %')
plt.xlabel('Credit Card limit')
plt.show()

In [ ]:
data["LIMIT_BAL"].value_counts().head(2)

From the plot shows that largest group with credit card limit is 50K group and second largest group of clients with credit card limit is 20K group. 

The credit card limit is a right tail distribution.

**Credit Card Limitation Distribution,Group by Gender** 

Now,we want to see is there a difference between people who is creaditable or not.

In [ ]:
class_0 = data.loc[data['default payment next month'] == 0]["LIMIT_BAL"]
class_1 = data.loc[data['default payment next month'] == 1]["LIMIT_BAL"]
plt.figure(figsize = (12,4))
sns.distplot(class_1,kde=True,bins=200, color="red")
sns.distplot(class_0,kde=True,bins=200, color="blue")
plt.title("Default amount of credit limit,Group by default payment next month")
plt.legend(["Default","Not Default"])
plt.xticks(range(0,1000000,250000))
plt.ylabel('Percentage %')
plt.xlabel('Credit Card limit')
plt.show()

Most of not creditable people have credit card limit around 20K to 50K. The clients without default payment have higher limit than people who with default payment. 

**Gender**

Secondly, we want to see it there have a gender discrimination with credit card limit.

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(x="SEX",y="LIMIT_BAL",data=data,hue="SEX")
plt.ylabel('Credit Card limit')
plt.xlabel("Gender")
plt.title("Default amount of credit card limit,Group by gender")
plt.show()


The plot shows that male and female have similar credit card limit, so that gender may not be a great variables to use in classification.

Female has a outler, but we think it is acceptable since maybe she is a CEO of some company and she has enough credit to loan more.

**Correlation**

Check the if there are linear relationships between features

In [ ]:
def plot_corr(df,size=15):
    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    cax = ax.matshow(corr,cmap=plt.get_cmap('rainbow'))
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.colorbar(cax)
plot_corr(data)


There are two main part of correlation, one is PAY variables and Bill variables.

Corrlation of payment status is decreasing by times go on. From current month to past month or next month have highest correlation.  The lowest correlation is between September and April. 

Also, corrlation of bill is decreasing by times go on. From current month to past month or next month have highest correlation. And the lowest correlation is between September and April. 

**Credit Card limitation,group by Age and Gender(Side-by-side Barplot)**

In [ ]:
fig, ax1 = plt.subplots(ncols=1, figsize=(16,6))
sns.boxplot(ax=ax1, x="AGE", y="LIMIT_BAL", hue="SEX",data=data)
plt.ylabel('Credit Card limit')
plt.xlabel("AGE")
plt.title("Default amount of credit card limit,group by age and sex")
plt.show()

Mean of credit card limit is increasing from age 21 to age 35. And after 35 years old, the mean of limit is remain stable and it's little bit decreasing from 40 to 60. The obvious difference between male and female is age 65 to age 79, those age have extreme large diffences, for example, at age 39,48 until 60, the mean value for males are much larger than female.

**Marriage**

We may thinking about the marriage is a main factor to effect a person who with/without default payment.

In [ ]:
sns.countplot(x="MARRIAGE", data=data,hue="default payment next month", palette="muted")

As the plot shows, marriage people have less default payment issue compare with single people.

It states that marriage status is a factor to effect a default payment.

**Data Modeling**

In [ ]:
#split the train data and test data
random.seed(777)
train_data = data.sample(frac=5/6, replace=False)
test_data = data.sample(frac=1/6, replace=False)
print (train_data.shape)
print (test_data.shape)

In [ ]:
y_train = train_data['default payment next month']
y_test=test_data['default payment next month']
x_train = train_data.drop(['default payment next month', 'ID'], axis= 1)
x_test = test_data.drop(['default payment next month', 'ID'], axis= 1)
print (y_train.shape)
print (y_test.shape)
print (x_train.shape)
print (x_test.shape)

We used random forest tree model as my first baseline model since:
1. By principle since it randomizes the variable selection during each tree split it's not prone to overfit unlike other models.
2. We don't need to pre-precessing data too much.
3. I used to read that RF don't need CV because it has includes bootstrap sampling and random variable selection in the process of model fit.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(class_weight = {0:1, 1:3})
RF.fit(x_train, y_train)
y_pred = RF.predict(x_test)
acc_RF = round(accuracy_score(y_test,y_pred) * 100, 2)
print(RF.score(x_train, y_train))
print (acc_RF)

In [ ]:
features = pd.DataFrame()
features['feature'] = x_train.columns
features['importance'] = RF.feature_importances_
features = features.sort_values(by='importance',ascending=False)
plt.figure(figsize = (7,4))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='feature',y='importance',data=features)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show() 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
def search_model(x_train, y_train, est, param_grid, n_jobs, cv):
    model = GridSearchCV(estimator  = est,
                                     param_grid = param_grid,
                                     scoring = 'f1_weighted',
                                     verbose = 10,
                                     n_jobs = n_jobs,
                                     iid = True,
                                     cv = cv)
    # Fit Grid Search Model
    model.fit(x_train, y_train)   
    return model

In [ ]:
## Grid search is actually using cross validation method to tune parameters, 
## which is to scan all posible parameter combinations and return the best one. 

In [ ]:
param_grid = {'n_estimators':[100,300,500],
             'criterion':['gini', 'entropy'],
             'class_weight': [{0:1, 1:3}]}

RF = search_model(x_train, y_train
            , RandomForestClassifier()
            , param_grid
            , -1
            , 5)
RF.fit(x_train, y_train)
y_pred = RF.predict(x_test)
acc_RF = round(accuracy_score(y_test,y_pred) * 100, 2)
print(RF.score(x_train, y_train))
print (acc_RF)

In [ ]:
#logistic
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred_log = model.predict(x_test)
acc_log = round(accuracy_score(y_test,y_pred_log) * 100, 2)
print (acc_log)

In [ ]:
#logisticCV
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV()
model.fit(x_train, y_train)
y_pred_logcv = model.predict(x_test)
acc_logcv = round(accuracy_score(y_test,y_pred_logcv) * 100, 2)
print (acc_logcv)

In [ ]:
#SVM
from sklearn.svm import SVC
model = SVC()
model.fit(x_train, y_train)
y_pred_svc = model.predict(x_test)
acc_svc = round(accuracy_score(y_test,y_pred_svc) * 100, 2)
print (acc_svc)

In [ ]:
#Linear SVM
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(x_train, y_train)
y_pred_lsvc = model.predict(x_test)
acc_lsvc = round(accuracy_score(y_test,y_pred_lsvc) * 100, 2)
print (acc_lsvc)

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 3)
model.fit(x_train, y_train)
y_pred_knn = model.predict(x_test)
acc_knn = round(accuracy_score(y_test,y_pred_knn) * 100, 2)
print (acc_knn)

In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred_DT = model.predict(x_test)
acc_DT = round(accuracy_score(y_test,y_pred_DT) * 100, 2)
print (acc_DT)

In [ ]:
#Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
y_pred_GNB = model.predict(x_test)
acc_GNB = round(accuracy_score(y_test,y_pred_GNB) * 100, 2)
print (acc_GNB)

In [ ]:
acc = pd.DataFrame()
acc['accuracy'] = [acc_RF,acc_log,acc_logcv,acc_svc,acc_lsvc,acc_knn,acc_DT,acc_GNB]
acc['method']=['Random Forester','Logistic','LogisticCV',"SVM",
               "Linear SVM","KNN","Decision Tree",'Gaussian Naive Bayes']
acc.index=acc.method
acc = acc.sort_values(by='accuracy',ascending=False)
print(acc.drop(['method'],axis=1))

In [ ]:
plt.figure(figsize = (7,4))
plt.title('Accuracy',fontsize=14)
s = sns.barplot(x='method',y='accuracy',data=acc)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show() 